In [2]:
import tempfile
import os
import uuid
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chat_models.gigachat import GigaChat
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
import logging 
import PyPDF2
import os
os.environ['CURL_CA_BUNDLE'] = ''
import warnings
warnings.filterwarnings("ignore")

In [38]:
llm =  GigaChat(auth_url='https://sm-auth-sd.prom-88-89-apps.ocp-geo.ocp.sigma.sbrf.ru/api/v2/oauth',
               credentials= 'ODRlZjMxYTQtZmQyNi00ZmUzLThhZjUtODEyNWRhMzE0ZGNiOjY1OTZhY2YyLThiZDgtNDI4Yi1hMDE1LTI0MGY5MzNmZGIwZA==', 
               verify_ssl_certs=False)

In [39]:
def get_embeddings():
    model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    model_kwargs = {'device': 'cpu'}
    # model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': False}
    return HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
        )

In [40]:
loader = PyPDFLoader("C:/Users/22456023/Desktop/GigaChat.pdf")
docs = loader.load()

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 45 0 (offset 0)
Ignoring wrong pointing object 47 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 63 0 (offset 0)
Ignoring wrong pointing object 65 0 (offset 0)


In [41]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import TextLoader

documents = []
for file in os.listdir('files'):
    if file.endswith('.pdf'):
        pdf_path = './files/' + file
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())
    elif file.endswith('.docx') or file.endswith('.doc'):
        doc_path = './files/' + file
        loader = Docx2txtLoader(doc_path)
        documents.extend(loader.load())
    elif file.endswith('.txt'):
        text_path = './files/' + file
        loader = TextLoader(text_path, encoding='utf-8')
        documents.extend(loader.load())

In [42]:
documents[:1]
get_embeddings()

No sentence-transformers model found with name sentence-transformers/paraphrase-multilingual-mpnet-base-v2. Creating a new one with MEAN pooling.


SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /sentence-transformers/paraphrase-multilingual-mpnet-base-v2/resolve/main/config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))"), '(Request ID: 6b5c66f8-714a-4e49-8773-4c4d9200dff8)')

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,)
splits = text_splitter.split_documents(documents[:1])
embedding = get_embeddings()
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)
retriever = vectorstore.as_retriever(search_kwargs={"k":3})

Error importing huggingface_hub.hf_file_system: module 'fsspec' has no attribute 'spec'
Error importing huggingface_hub.hf_file_system: module 'fsspec' has no attribute 'spec'


ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [ ]:
store = {}

In [ ]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [ ]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
output=conversational_rag_chain.invoke(
    {"input": "Какой суммой ограничена ответственность банка?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  
)["answer"]

In [ ]:
print (output)

Ответственность банка ограничена суммой 1 000 рублей.


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_pickle('ru_rag_test_dataset.pkl')
df

,Вопрос,Правильный ответ,Контекст,Файл
0,Что может вызвать цунами?,Землетрясение,"Землетрясения, извержения вулканов и другие по...",19985.txt
1,Кто написал роман «Хижина дяди Тома»?,Г. Бичер-Стоу,«Хижина дяди Тома» (англ. Uncle Tom's Cabin) —...,273921.txt
2,Кто создал Зимний дворец в Санкт-Петербурге?,Растрелли,Зимний дворец (в 1918—1943 годах — Дворец Иску...,50.txt
3,Какое современное государство занимает террито...,Ирак,"Нине́ви́я или Нин (аккад. 𒌷𒉌𒉡𒀀, Ninwe; ассир. ...",36300.txt
4,Как звали отца Северуса Снейпа?,Тоббиас Снегг (Снейп),Происхождение Снегга в основном стало известно...,172300.txt
...,...,...,...,...
918,На какой реке стоит Череповец?,Шексна,Пятый гидроузел — Шекснинский (Череповецкий) Ш...,42745.txt
919,Какая страна является победителем Чемпионата м...,Германия,Магдале́на Но́йнер (нем. Magdalena Neuner; род...,400782.txt
920,Кто спроектировал Mercedes-AMG GT?,"Роберт Лесник, Марк Фетерстон, Ян Кауль",Автомобиль Mercedes-Benz SLS AMG имеет длину в...,1989102.txt
921,Какова высота Биг Бена?,96,"Высота башни, включая шпиль — 96 метров (315 ф...",329717.txt
